In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from pathlib import Path
import textwrap
import cga_utils
import math
from typing import Iterable, List, Optional, Tuple
import hdbscan
from sklearn.feature_extraction import DictVectorizer
import cluster_features
%load_ext autoreload
%autoreload 2

# Used in the paper

In [ ]:
len(pd.read_csv('res/gemma3_v18.csv'))

In [11]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

EPS = 1e-9

def value_bucket(ans, pred):
    if ans == pred:
        return 0
    if ans == -1*pred:
        return 1
    if math.isnan(pred):
        return 2
    else: 
        return 3


errors = pd.read_csv('res/gemma3_v18.csv').query('exact_match == False')
print("Error set n:", len(errors), ", Q1: ", len(errors)/4)
# extra features
errors["ratio"] = errors.apply(lambda row: (row["pred"] / (row["answer"]+EPS)) if row["answer"] != 0 else np.inf,  axis=1 )
errors["x100_flag"] = errors["ratio"].apply(lambda ratio: int(0.95 < ratio/100 < 1.05 or 0.95 < ratio*100 < 1.05))

errors["scale"] = errors["scale"].fillna("")
errors["pred_scale"] = errors["pred_scale"].fillna("")
errors["code_calc_pattern"]= errors["code_calc_pattern"].fillna("nan")

errors["scale_mismatch"]= errors["scale"] != errors["pred_scale"]
errors["value_mismatch"]= errors["answer"] != errors["pred"]
errors["value_nan"]= errors["pred"].isna()
errors["value_match"]= errors.apply(lambda row: value_bucket(row["answer"], row["pred"]) ,  axis=1 )

errors["calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["calc_pattern"],3) 
errors["code_calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["code_calc_pattern"],3) 
#convert
#vects = errors[["calc_pattern", "code_calc_pattern", "scale", "pred_scale",'sign_error', "x100_flag"]]
vects = errors[["calc_pattern", "code_calc_pattern", "scale_mismatch", "value_match", "error_code"]]

X_dict = vects.to_dict(orient="records")
dv = DictVectorizer(sparse=False)

X_features = dv.fit_transform(X_dict).astype(int)

setups = cluster_features.try_hdbscan(X_features)
print(setups)
#print([ (a,(b,c,d,e)) for (a,b) in setups.iterrows()])
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, metric="hamming")
clusterer.fit_predict(X_features)

errors["cluster"] = clusterer.labels_

counts = errors["cluster"].value_counts()
print(counts)
errors.to_csv('res/errors_18_cv1_2_1.csv')
for id in counts.index[:]:    
#errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query('cluster == cluster[cluster != -1].value_counts().idxmax()')    
    print(id)
    print(errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == {id}'))
len (errors)

Error set n: 285 , Q1:  71.25
merging
merging
    min_cluster_size  min_samples  n_clusters  noise_ratio
30                15            1           3     0.038596
31                15            2           2     0.052632
32                15            3           2     0.066667
33                15            4           2     0.108772
0                  2            1          65     0.119298
34                15            5           2     0.133333
5                  3            1          46     0.157895
25                10            1           9     0.182456
1                  2            2          50     0.185965
10                 4            1          31     0.214035
6                  3            2          42     0.217544
11                 4            2          27     0.277193
2                  2            3          41     0.294737
3                  2            4          41     0.305263
15                 5            1          22     0.305263
20        

285

In [12]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

EPS = 1e-9

def value_bucket(ans, pred):
    if ans == pred:
        return 0
    if ans == -1*pred:
        return 1
    if math.isnan(pred):
        return 2
    else: 
        return 3


errors = pd.read_csv('res/gemma3_v19a.csv').query('exact_match == False')
print("Error set n:", len(errors), ", Q1: ", len(errors)/4)
# extra features
errors["ratio"] = errors.apply(lambda row: (row["pred"] / (row["answer"]+EPS)) if row["answer"] != 0 else np.inf,  axis=1 )
errors["x100_flag"] = errors["ratio"].apply(lambda ratio: int(0.95 < ratio/100 < 1.05 or 0.95 < ratio*100 < 1.05))

errors["scale"] = errors["scale"].fillna("")
errors["pred_scale"] = errors["pred_scale"].fillna("")
errors["code_calc_pattern"]= errors["code_calc_pattern"].fillna("nan")

errors["scale_mismatch"]= errors["scale"] != errors["pred_scale"]
errors["value_mismatch"]= errors["answer"] != errors["pred"]
errors["value_nan"]= errors["pred"].isna()
errors["value_match"]= errors.apply(lambda row: value_bucket(row["answer"], row["pred"]) ,  axis=1 )

errors["calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["calc_pattern"],3) 
errors["code_calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["code_calc_pattern"],3) 
#convert
#vects = errors[["calc_pattern", "code_calc_pattern", "scale", "pred_scale",'sign_error', "x100_flag"]]
vects = errors[["calc_pattern", "code_calc_pattern", "scale_mismatch", "value_match", "error_code"]]

X_dict = vects.to_dict(orient="records")
dv = DictVectorizer(sparse=False)

X_features = dv.fit_transform(X_dict).astype(int)

setups = cluster_features.try_hdbscan(X_features)
print(setups)
#print([ (a,(b,c,d,e)) for (a,b) in setups.iterrows()])
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, metric="hamming")
clusterer.fit_predict(X_features)

errors["cluster"] = clusterer.labels_

counts = errors["cluster"].value_counts()
print(counts)
errors.to_csv('res/errors_19a_cv1_2_1.csv')
for id in counts.index[:]:    
#errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query('cluster == cluster[cluster != -1].value_counts().idxmax()')    
    print(id)
    print(errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == {id}'))
len (errors)

Error set n: 266 , Q1:  66.5
merging
merging
    min_cluster_size  min_samples  n_clusters  noise_ratio
30                15            1           3     0.086466
31                15            2           2     0.097744
32                15            3           2     0.124060
0                  2            1          62     0.142857
33                15            4           2     0.195489
5                  3            1          38     0.195489
34                15            5           2     0.203008
1                  2            2          44     0.214286
10                 4            1          29     0.221805
6                  3            2          36     0.259398
15                 5            1          22     0.270677
2                  2            3          40     0.281955
25                10            1          12     0.296992
4                  2            5          39     0.319549
3                  2            4          37     0.334586
11         

266

In [6]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 28').to_csv('res/gemma3_alg_step2.csv')

In [7]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 60').to_csv('res/gemma3_alg_step2b.csv')

In [23]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

EPS = 1e-9

def value_bucket(ans, pred):
    if ans == pred:
        return 0
    if ans == -1*pred:
        return 1
    if math.isnan(pred):
        return 2
    else: 
        return 3


errors = pd.read_csv('res/gemma3_v21.csv').query('exact_match == False')
print("Error set n:", len(errors), ", Q1: ", len(errors)/4)
# extra features
errors["ratio"] = errors.apply(lambda row: (row["pred"] / (row["answer"]+EPS)) if row["answer"] != 0 else np.inf,  axis=1 )
errors["x100_flag"] = errors["ratio"].apply(lambda ratio: int(0.95 < ratio/100 < 1.05 or 0.95 < ratio*100 < 1.05))

errors["scale"] = errors["scale"].fillna("")
errors["pred_scale"] = errors["pred_scale"].fillna("")
errors["code_calc_pattern"]= errors["code_calc_pattern"].fillna("nan")

errors["scale_mismatch"]= errors["scale"] != errors["pred_scale"]
errors["value_mismatch"]= errors["answer"] != errors["pred"]
errors["value_nan"]= errors["pred"].isna()
errors["value_match"]= errors.apply(lambda row: value_bucket(row["answer"], row["pred"]) ,  axis=1 )

errors["calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["calc_pattern"],3) 
errors["code_calc_pattern"] = cga_utils.aggregate_calc_patterns(errors["code_calc_pattern"],3) 
#convert
#vects = errors[["calc_pattern", "code_calc_pattern", "scale", "pred_scale",'sign_error', "x100_flag"]]
vects = errors[["calc_pattern", "code_calc_pattern", "scale_mismatch", "value_match", "error_code"]]

X_dict = vects.to_dict(orient="records")
dv = DictVectorizer(sparse=False)

X_features = dv.fit_transform(X_dict).astype(int)

setups = cluster_features.try_hdbscan(X_features)
print(setups)
#print([ (a,(b,c,d,e)) for (a,b) in setups.iterrows()])
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, metric="hamming")
clusterer.fit_predict(X_features)

errors["cluster"] = clusterer.labels_

counts = errors["cluster"].value_counts()
print(counts)
errors.to_csv('res/errors_21_cv1_2_1.csv')
for id in counts.index[:]:    
#errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query('cluster == cluster[cluster != -1].value_counts().idxmax()')    
    print(id)
    print(errors[["question","calc_pattern","code_calc_pattern","derivation", "answer","pred" ,'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == {id}'))
len (errors)

Error set n: 267 , Q1:  66.75
merging
merging
    min_cluster_size  min_samples  n_clusters  noise_ratio
0                  2            1          61     0.112360
31                15            2           7     0.157303
1                  2            2          43     0.202247
5                  3            1          37     0.213483
21                 6            2          18     0.228464
16                 5            2          19     0.228464
26                10            2          12     0.235955
6                  3            2          35     0.250936
10                 4            1          21     0.269663
11                 4            2          21     0.269663
2                  2            3          34     0.280899
15                 5            1          18     0.299625
17                 5            3          17     0.307116
34                15            5           5     0.307116
3                  2            4          37     0.318352
20        

267

In [26]:
#errors[["question","calc_pattern","code_calc_pattern",'scale', 'pred_scale', 'sign_error', "x100_flag", "error_code",  "cluster"]].query(f'cluster == 37').to_csv('res/gemma3_alg_step1.csv')
errors.query(f'cluster == 59').to_csv('res/gemma3_alg_step3.csv')